In [2]:
import pandas as pd
from pathlib import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

### BASIC DATA CLEANING

- Try Understanding the meaning of each file and creating a scenario out of it
- Its simple, a patient goes to a hospital and is attended by a healthcare provider. The patient is then treated for some given ailment. Certain medications and procedures are prescribed to the patient and most of the cost is covered by the patient's Insurance agency 

- Now try relating this to each of the csv files

- Patient.csv  ---> the patient details

- Organisation.csv ---> the hospital details

- Providers.csv ---> healthcare professional details

- payers.csv ---> Insurance agency details

- medications.csv ---> medicines and drug details

- procedures.csv ---> procedures / medical tests 

- devices.csv ---> devices used

- encounters.csv ---> the entire event itself

#### The encounter data is the crux for all the other data. Let's take a look at it

In [5]:
encounter=pd.read_csv("../data/encounters.csv")
encounter.sample(3)

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
39555,cc40d248-15b0-45e2-9eb5-40cdf536c6db,2020-03-06T04:54:49Z,2020-03-06T05:24:49Z,8bd293c5-cde8-4ab0-b96c-9591f372de4e,5aa98f16-c832-3ec5-b52a-2691978cc235,7a9bf835-c4e3-35cd-89c8-216837ff747b,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,410620009,Well child visit (procedure),129.16,129.16,129.16,NaN,NaN
42335,11123558-3305-49e5-9bf3-210b4cb07404,2007-02-09T07:37:41Z,2007-02-09T08:07:41Z,d78faf3a-dfad-4b6b-a353-e688cd2b8568,42bab40f-8586-3ff2-820b-5f70251d3068,75ba00e1-f758-306d-872d-52a41f7c7dd6,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,wellness,162673000,General examination of patient (procedure),129.16,129.16,129.16,NaN,NaN
31348,ae250109-b2c0-4821-a6cc-f1632e7d0451,2005-02-21T23:35:03Z,2005-02-21T23:50:03Z,cae10920-f977-48b4-a0d3-4d70ad561fd1,be4c63f3-8d38-3fa9-a183-62045b5c85f6,8f9aea5b-fd01-37c0-8931-18b6d64bdae6,7caa7254-5050-3b5e-9eae-bd5ea30e809c,ambulatory,185347001,Encounter for problem (procedure),129.16,129.16,89.16,NaN,NaN


#### We need only a select number of columns. Also how about renaming them to be a bit descriptive

All column names in caps

patient ---> patient_id

organisation ---> org_id

provider ---> provider_id

payer ---> payer_id

reasondescription --->reason


include the columns
["Id","START","STOP","PATIENT_ID","ORG_ID","PROVIDER_ID",
"PAYER_ID","DESCRIPTION","REASON","BASE_ENCOUNTER_COST",
"PAYER_COVERAGE","ENCOUNTERCLASS"]

In [7]:
encounter=encounter.rename(columns={"PATIENT":"PATIENT_ID","ORGANIZATION":"ORG_ID","PROVIDER":"PROVIDER_ID","PAYER":"PAYER_ID","REASONDESCRIPTION":"REASON"})
encounter=encounter[["Id","START","STOP","PATIENT_ID","ORG_ID","PROVIDER_ID",
"PAYER_ID","DESCRIPTION","REASON","BASE_ENCOUNTER_COST",
"PAYER_COVERAGE","ENCOUNTERCLASS"]]
encounter.sample(3)

,Id,START,STOP,PATIENT_ID,ORG_ID,PROVIDER_ID,PAYER_ID,DESCRIPTION,REASON,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ENCOUNTERCLASS
36700,cd93aed0-5d4e-4567-bd9c-2f4701d01b5a,2019-07-29T13:04:59Z,2019-07-29T13:19:59Z,e073858e-b506-4849-9cac-f764cd9218c2,240b2584-02de-3b5c-af58-4c0a84b248ca,a43b394d-7279-36fd-9856-c304ff67c754,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,General examination of patient (procedure),NaN,129.16,129.16,wellness
33408,cc796301-20c1-4d74-8b1a-769d9e1a1319,2018-09-05T23:09:30Z,2018-09-05T23:24:30Z,0df7cd60-097e-4fba-90eb-f7e9d9de0b67,b81688f5-bd0e-3c99-963f-860d3e90ab5d,7572448e-118a-3102-84ac-a5e4281a5c2d,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,Urgent care clinic (procedure),NaN,129.16,0.00,urgentcare
25642,49e0473f-c008-4dfd-802d-addd6a856ce7,1996-05-17T16:00:12Z,1996-05-17T16:15:12Z,6aa4b3c1-4b16-479e-a618-0fc47fb99f6e,7834877b-2072-3d79-9137-d5ee86c7503f,8daf994f-712d-3f7f-816a-71cb6766510e,6e2f1a2d-27bd-3701-8d08-dae202c58632,General examination of patient (procedure),NaN,129.16,54.16,wellness


#### Now lets connect all the other elements and make one coherent data source

### I -  PATIENTS

In [8]:
patients=pd.read_csv("../data/patients.csv")
patients.sample(3)

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
927,dc811097-b472-4dbd-8ce9-bb0afca44d7e,1995-02-18,NaN,999-19-6566,S99934669,X79127078X,Mr.,Alan320,Tillman293,NaN,...,Plymouth Massachusetts US,563 Hansen Ranch,New Bedford,Massachusetts,Bristol County,2744.0,41.762338,-70.902124,610115.95,3293.89
724,81813194-e1ed-41c2-8f7a-122a60b06550,1971-04-30,NaN,999-75-5931,S99965713,X66224540X,Mrs.,Merideth332,Littel644,NaN,...,Milton Massachusetts US,744 Lehner Overpass Suite 71,Rockport,Massachusetts,Essex County,NaN,42.697827,-70.652358,1054359.51,9522.78
979,71257193-1eb5-45e4-9a86-37b8f5d56340,1963-01-06,NaN,999-93-7616,S99981961,X36701322X,Mrs.,Melinda114,Price929,NaN,...,Raynham Massachusetts US,214 Gulgowski Arcade,Arlington,Massachusetts,Middlesex County,2476.0,42.396815,-71.114405,1366522.35,12549.07


#### Lets do some basic cleaning:

- Select only a few important columns
- Calculate the age of the patients
- Remove outliers (Consider assumption that age is < 100)
- Join the Patients first and last name

#### columns needed : ["Id","BIRTHDATE","FIRST","LAST","CITY"]


In [9]:
# SELECT ONLY REQUIRED COLUMNS
patients=patients[["Id","BIRTHDATE","FIRST","LAST","CITY"]]
patients.head()

,Id,BIRTHDATE,FIRST,LAST,CITY
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,José Eduardo181,Gómez206,Chicopee
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,Milo271,Feil794,Somerville
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,Jayson808,Fadel536,Chicopee
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,Mariana775,Rutherford999,Lowell
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,Gregorio366,Auer97,Boston


In [10]:
# CREATE A COLUMN AGE THAT CAN BE CALCULATED USING BIRTHDATE

from datetime import datetime

current_year= datetime.today().year

patients["BIRTHDATE"]=pd.to_datetime(patients["BIRTHDATE"],format= "%Y-%m-%d")

patients["AGE"]=patients["BIRTHDATE"].apply(lambda x: current_year - x.year)


# SELECT ROWS WHERE AGE < 100 AND DROP BIRTHDATE COLUMN

patients = patients.loc[patients["AGE"]<100]

patients = patients.rename(columns={"Id":"PATIENT_ID"})

patients = patients.set_index("PATIENT_ID")

patients.sample(3)

,BIRTHDATE,FIRST,LAST,CITY,AGE
PATIENT_ID,,,,,
ca8803ac-66ef-4895-a8c4-290313fcee6f,1980-05-06,Rowena386,Borer986,Somerville,40
4bbde047-119f-4a12-9b68-2a89b274364e,2012-09-09,Alia555,Rohan584,North Reading,8
9c69f2d5-b7de-4634-84c5-bf3efd551f5e,1962-05-15,Danial835,D'Amore443,Peabody,58


In [11]:

# CREATE A COLUMN CALLED 'PATIENT_NAME' THAT CONCATS THE FIRST AND LAST NAME

patients["PATIENT_NAME"]=patients["FIRST"].str.cat(patients["LAST"],sep=" ")

#DROP FIRST AND LAST NAME COLUMNS

patients = patients.drop(["FIRST","LAST"],axis=1)

patients.sample(3)

,BIRTHDATE,CITY,AGE,PATIENT_NAME
PATIENT_ID,,,,
09fc03c4-aef7-47e1-941a-618f437dfbe2,1991-06-18,Lawrence,29,Elijah719 Zboncak558
532696f2-0b76-4eb0-9aea-a74e2fb1bed2,1967-05-18,Natick,53,Takisha161 Kunze215
65fdd955-9b96-4faa-8b71-c29b8807f709,1986-02-01,Boston,34,Esteban536 Gorczany269


### Now let's integrate this with the encounter data

- We're Using 'join' in the this. By the way do you know whats the difference between join and merge ??

In [12]:
encounter=encounter.join(patients, on = "PATIENT_ID")
encounter.sample(3)

,Id,START,STOP,PATIENT_ID,ORG_ID,PROVIDER_ID,PAYER_ID,DESCRIPTION,REASON,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ENCOUNTERCLASS,BIRTHDATE,CITY,AGE,PATIENT_NAME
48752,cdb24781-ee3c-4a6e-8953-4c42ae71f9e5,2012-03-03T10:53:35Z,2012-03-03T11:38:35Z,ebee7497-67ea-4a51-ade2-59d7dfca8fcf,5103c940-0c08-392f-95cd-446e0cea042a,d78462d1-3e5d-3a93-961e-6df1df9e285a,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,Patient-initiated encounter,Normal pregnancy,129.16,59.16,ambulatory,1981-08-08,Springfield,39.0,Latoyia537 Schaden604
21546,f13c9b0a-fb71-47ed-a499-1e8f679a4450,2014-02-26T01:48:55Z,2014-02-26T02:03:55Z,7148eb24-d1f0-4a5c-93c0-3d2a632f25da,a311cff8-ee06-3573-ba5a-42e96e52b0fb,a72dbca3-20ca-357e-a7be-6f09ff833a9a,4d71f845-a6a9-3c39-b242-14d25ef86a8d,General examination of patient (procedure),NaN,129.16,129.16,wellness,1962-02-13,Boston,58.0,Orval846 Gaylord332
1041,32190289-1002-483d-bc02-f75c9eacb93a,2016-06-07T20:46:46Z,2016-06-07T21:34:46Z,bf35e4fa-ea4f-40a4-8fe6-1f2f26e0aa45,24cb4eab-6166-3530-bddc-a5a8a14a4fc1,7bd4e666-a82d-3ad1-bc7c-b49eb726577b,047f6ec3-6215-35eb-9608-f9dda363a44c,Outpatient procedure,NaN,129.16,69.16,ambulatory,2000-11-21,Ashland,20.0,Cecila397 Feil794


####  II HOSPITALS

In [13]:
# READ THE ORGANISATION FILES

organisations=pd.read_csv("../data/organizations.csv")
organisations.sample(3)

,Id,NAME,ADDRESS,CITY,STATE,ZIP,LAT,LON,PHONE,REVENUE,UTILIZATION
993,903c4e22-78f4-36e7-b4f0-e4078fb8b63c,PERFORMANCE REHABILITATION OF WESTERN NEW ENGL...,11 CABOT RD,WOBURN,MA,01801-1003,42.488769,-71.154438,NaN,3229.0,25
306,2f09e5c4-f013-3b8e-aa9d-6564542502a0,PCP31724,300 NEWBURYPORT TRUNPIKE,ROWLEY,MA,01969,42.721495,-70.886637,978-948-3900,3874.8,30
614,cd78e27b-c9fa-3144-80b5-c67a7bf4ceca,PCP139247,46 COLUMBIA RD,PEMBROKE,MA,02359-1862,42.067804,-70.805713,NaN,3229.0,25


In [16]:
# SELECT A FEW COLUMNS AND RENAME THEM
# COLUMN NAMES : ["Id","NAME","CITY","REVENUE"]

organisations = organisations[["Id","NAME","CITY","REVENUE"]]

# RENAME NAME --> ORG_NAME, Id --> ORG_ID , CITY ---> ORG_CITY

organisations = organisations.rename(columns={"NAME":"ORG_NAME","Id":"ORG_ID","CITY":"ORG_CITY"})

#SET INDEX AS ORG_ID

organisations = organisations.set_index("ORG_ID")

organisations.sample(3) 

,ORG_NAME,ORG_CITY,REVENUE
ORG_ID,,,
ea3c261b-50d8-30a0-917c-7a31f2862e97,PCP211168,JAMAICA PLAIN,11366.08
e9e2998f-97d3-3f88-8ee8-82660c5ddfce,PCP128586,W WAREHAM,9557.84
e613f5a1-7588-3efe-8701-fa9baf3241ff,PCP31073,REHOBOTH,15111.72


In [17]:
# JOIN TO  ENCOUNTERS
encounter = encounter.join(organisations,on="ORG_ID")
encounter

,Id,START,STOP,PATIENT_ID,ORG_ID,PROVIDER_ID,PAYER_ID,DESCRIPTION,REASON,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ENCOUNTERCLASS,BIRTHDATE,CITY,AGE,PATIENT_NAME,ORG_NAME,ORG_CITY,REVENUE
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23T17:45:28Z,2010-01-23T18:10:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,6e2f1a2d-27bd-3701-8d08-dae202c58632,Encounter for symptom,Acute bronchitis (disorder),129.16,54.16,ambulatory,1983-11-14,Somerville,37.0,Milo271 Feil794,CAMBRIDGE HEALTH ALLIANCE,CAMBRIDGE,406053.19
1,e88bc3a9-007c-405e-aabc-792a38f4aa2b,2012-01-23T17:45:28Z,2012-01-23T18:00:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,772ee193-bb9f-30eb-9939-21e86c8e4da5,6f1d59a7-a5bd-3cf9-9671-5bad2f351c28,6e2f1a2d-27bd-3701-8d08-dae202c58632,General examination of patient (procedure),NaN,129.16,129.16,wellness,1983-11-14,Somerville,37.0,Milo271 Feil794,PCP20565,CAMBRIDGE,4262.28
2,8f104aa7-4ca9-4473-885a-bba2437df588,2001-05-01T15:02:18Z,2001-05-01T15:17:18Z,1d604da9-9a81-4ba9-80c2-de3375d59b40,5d4b9df1-93ae-3bc9-b680-03249990e558,af01a385-31d3-3c77-8fdb-2867fe88df2f,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,Encounter for symptom,Sinusitis (disorder),129.16,0.00,ambulatory,1989-05-25,Chicopee,31.0,José Eduardo181 Gómez206,HOLYOKE MEDICAL CENTER,HOLYOKE,327601.37
3,b85c339a-6076-43ed-b9d0-9cf013dec49d,2011-07-28T15:02:18Z,2011-07-28T15:17:18Z,1d604da9-9a81-4ba9-80c2-de3375d59b40,3dc9bb2d-5d66-3e61-bf9a-e234c6433577,bb17e691-262b-3546-93d5-d88e7de93246,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,General examination of patient (procedure),NaN,129.16,0.00,wellness,1989-05-25,Chicopee,31.0,José Eduardo181 Gómez206,PCP12399,SOUTH HADLEY,7491.28
4,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,2010-07-27T12:58:08Z,2010-07-27T13:28:08Z,10339b10-3cd1-4ac3-ac13-ec26728cb592,b03dba4f-892f-365c-bfd1-bfcfa7a98d5d,7ed6b84a-b847-3744-9d42-15c42297a0c2,d47b3510-2895-3b70-9897-342d681c769d,General examination of patient (procedure),NaN,129.16,129.16,wellness,1992-06-02,Chicopee,28.0,Jayson808 Fadel536,WESTERN MASS PHYSICIAN ASSOCIATES INC,CHICOPEE,7620.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53341,bd3bfc3c-4f52-4d9c-aa01-60a084b2ddcd,2015-06-20T17:27:52Z,2015-06-20T17:57:52Z,fc817953-cc8b-45db-9c85-7c0ced8fa90d,a44b37da-aa9b-37f0-993e-26e58974e086,ff678a28-0ee8-379c-80ab-2dc790cf67dd,7caa7254-5050-3b5e-9eae-bd5ea30e809c,General examination of patient (procedure),NaN,129.16,129.16,wellness,NaT,NaN,NaN,NaN,PCP27951,WESTWOOD,12141.04
53342,59727285-d30f-44e2-bcda-eac21ce07269,2016-06-25T17:27:52Z,2016-06-25T17:42:52Z,fc817953-cc8b-45db-9c85-7c0ced8fa90d,a44b37da-aa9b-37f0-993e-26e58974e086,ff678a28-0ee8-379c-80ab-2dc790cf67dd,7caa7254-5050-3b5e-9eae-bd5ea30e809c,General examination of patient (procedure),NaN,129.16,129.16,wellness,NaT,NaN,NaN,NaN,PCP27951,WESTWOOD,12141.04
53343,957b13d5-49fa-4518-8c25-6eaa45e6a7d1,2017-07-01T17:27:52Z,2017-07-01T17:57:52Z,fc817953-cc8b-45db-9c85-7c0ced8fa90d,a44b37da-aa9b-37f0-993e-26e58974e086,ff678a28-0ee8-379c-80ab-2dc790cf67dd,7caa7254-5050-3b5e-9eae-bd5ea30e809c,General examination of patient (procedure),NaN,129.16,129.16,wellness,NaT,NaN,NaN,NaN,PCP27951,WESTWOOD,12141.04
53344,5529a0d5-a24c-42b4-a32a-729e4d1f19ef,2018-07-07T17:27:52Z,2018-07-07T17:42:52Z,fc817953-cc8b-45db-9c85-7c0ced8fa90d,a44b37da-aa9b-37f0-993e-26e58974e086,ff678a28-0ee8-379c-80ab-2dc790cf67dd,7caa7254-5050-3b5e-9eae-bd5ea30e809c,General examination of patient (procedure),NaN,129.16,129.16,wellness,NaT,NaN,NaN,NaN,PCP27951,WESTWOOD,12141.04


#### III HEALTHCARE PROFESSIONALS

In [18]:
providers=pd.read_csv("../data/providers.csv")
providers.sample(3)

,Id,ORGANIZATION,NAME,GENDER,SPECIALITY,ADDRESS,CITY,STATE,ZIP,LAT,LON,UTILIZATION
2864,ea97de5f-f95b-3b46-b301-5bcb3ada24bf,e26e0483-1a6b-3357-b042-3cdf1d61c612,Nicky270 Roberts511,M,GENERAL PRACTICE,33 VILLAGE SQ,CHELMSFORD,MA,01824-2712,42.597055,-71.369700,28
3363,773e4349-ee3a-356c-a116-e76759e3a527,add376a3-c999-349e-809d-1b7424405321,Sofia418 Cantú190,F,PHYSICAL THERAPY,176 WALKER ST,LOWELL,MA,01854-3126,42.638980,-71.321141,0
3305,80748a7e-3170-3a5b-a8da-7def919079d3,d0b5fc59-28d4-3a54-aaca-c93a3f1d3007,Guillermo498 Llamas954,M,OPTOMETRY,479 OLD UNION TURNPIKE,LANCASTER,MA,01523-3029,42.483569,-71.675585,0


In [19]:
# SELECT COLUMNS ["Id","NAME","GENDER","SPECIALITY"]
# RENAME COLUMNS NAME --> PROVIDER_NAME, Id --> PROVIDER_ID
# SET INDEX AS PROVIDER ID
providers = providers[["Id","NAME","GENDER","SPECIALITY"]]

providers = providers.rename(columns={"Id":"PROVIDER_ID","NAME":"PROVIDER_NAME"})

providers = providers.set_index("PROVIDER_ID")

providers.sample(3)

,PROVIDER_NAME,GENDER,SPECIALITY
PROVIDER_ID,,,
efebe8aa-ce46-3d28-a983-16180e7f8a06,Meri607 Lang846,F,ORTHOPEDIC SURGERY
6f0a80af-766f-315a-8804-277af9d23a8d,Israel728 Conn188,M,PSYCHIATRY
69f3f629-358b-3626-a647-ffd6f68a1b39,Vilma125 Becker968,F,CLINICAL SOCIAL WORKER


In [20]:
# join provider and hospitals
encounter = encounter.join(providers,on="PROVIDER_ID")
encounter.sample(3)

,Id,START,STOP,PATIENT_ID,ORG_ID,PROVIDER_ID,PAYER_ID,DESCRIPTION,REASON,BASE_ENCOUNTER_COST,...,BIRTHDATE,CITY,AGE,PATIENT_NAME,ORG_NAME,ORG_CITY,REVENUE,PROVIDER_NAME,GENDER,SPECIALITY
17353,afbf9fea-2c82-4614-91a9-f6ddf5398b62,1990-01-02T21:00:08Z,1990-01-02T21:30:08Z,70794fdc-0702-4e0f-9682-8ec5d7145436,0ad49cb1-6abc-34e1-a94c-ea3050595660,d4c730bc-c8f2-3428-8b90-9ac191bf2575,6e2f1a2d-27bd-3701-8d08-dae202c58632,General examination of patient (procedure),NaN,129.16,...,1940-05-14,Newton,80.0,Cristobal567 Upton904,PCP64655,WATERTOWN,11236.92,Judie557 Bergstrom287,F,GENERAL PRACTICE
40969,488fb252-eab5-4080-a0a0-257ab7e1f121,2019-03-22T20:51:21Z,2019-03-22T21:06:21Z,2c71dd97-7085-416a-aa07-d675bbe3adf2,4e916854-3a59-3f17-bb16-7c406148d822,93f77048-5bfd-3a59-9ae4-4399fbd77398,7caa7254-5050-3b5e-9eae-bd5ea30e809c,Urgent care clinic (procedure),NaN,129.16,...,1941-02-14,Hanover,79.0,Samatha845 Mueller846,HANOVER FAMILY HEALTH AND URGENT CARE,HANOVER,0.00,Tiana543 Blick895,F,GENERAL PRACTICE
39773,f304b206-6ba4-4053-b25c-b3ad513d79bf,2014-10-31T23:42:48Z,2014-11-01T00:12:48Z,f874e5d6-8e2e-453d-b054-a3b5770c4cdc,91a53589-2df1-3e27-a3eb-28970c71ae03,138f5f81-b364-3744-bfe5-6af816d1d266,4d71f845-a6a9-3c39-b242-14d25ef86a8d,Well child visit (procedure),NaN,129.16,...,1997-09-12,New Bedford,23.0,Douglass930 Pollich983,PCP31320,NEW BEDFORD,3745.64,Carlton317 Spencer878,M,GENERAL PRACTICE


#### IV  - INSURANCE COMPANIES

In [22]:
payers=pd.read_csv("../data/payers.csv")
payers.sample(3)

,Id,NAME,ADDRESS,CITY,STATE_HEADQUARTERED,ZIP,PHONE,AMOUNT_COVERED,AMOUNT_UNCOVERED,REVENUE,...,UNCOVERED_ENCOUNTERS,COVERED_MEDICATIONS,UNCOVERED_MEDICATIONS,COVERED_PROCEDURES,UNCOVERED_PROCEDURES,COVERED_IMMUNIZATIONS,UNCOVERED_IMMUNIZATIONS,UNIQUE_CUSTOMERS,QOLS_AVG,MEMBER_MONTHS
0,b3221cfc-24fb-339e-823d-bc4136cbc4ed,Dual Eligible,7500 Security Blvd,Baltimore,MD,21244.0,1-877-267-2323,141676.87,119449.83,1305000.0,...,0,556,0,280,0,223,0,25,0.362810,3348
1,7caa7254-5050-3b5e-9eae-bd5ea30e809c,Medicare,7500 Security Blvd,Baltimore,MD,21244.0,1-800-633-4227,4756720.13,890394.37,7122250.0,...,0,13471,0,7383,0,2557,0,198,0.786223,29760
8,42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,Anthem,220 Virginia Ave,Indianapolis,IN,46204.0,1-800-331-1476,526943.60,258995.00,152248800.0,...,7422,0,3912,2841,10142,6664,151,283,0.932301,69816


In [23]:
# COLUMNS NEEDED : NAME, Id
# RENAME NAME --> PAYER_NAME, Id---> PAYER_ID
# SET INDEX AS PAYER_ID

payers=payers[["NAME",'Id']]

payers=payers.rename(columns={"NAME":"PAYER_NAME","Id":"PAYER_ID"})

payers=payers.set_index("PAYER_ID")

payers

,PAYER_NAME
PAYER_ID,
b3221cfc-24fb-339e-823d-bc4136cbc4ed,Dual Eligible
7caa7254-5050-3b5e-9eae-bd5ea30e809c,Medicare
7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,Medicaid
d47b3510-2895-3b70-9897-342d681c769d,Humana
6e2f1a2d-27bd-3701-8d08-dae202c58632,Blue Cross Blue Shield
5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,UnitedHealthcare
4d71f845-a6a9-3c39-b242-14d25ef86a8d,Aetna
047f6ec3-6215-35eb-9608-f9dda363a44c,Cigna Health
42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,Anthem


In [25]:
# JOIN ON ENCOUNTERS
encounter = encounter.join(payers,on="PAYER_ID")

encounter.sample(3)

,Id,START,STOP,PATIENT_ID,ORG_ID,PROVIDER_ID,PAYER_ID,DESCRIPTION,REASON,BASE_ENCOUNTER_COST,...,CITY,AGE,PATIENT_NAME,ORG_NAME,ORG_CITY,REVENUE,PROVIDER_NAME,GENDER,SPECIALITY,PAYER_NAME
12938,78cda336-53df-41d6-a420-783907534933,2014-10-12T02:48:51Z,2014-10-12T03:03:51Z,0b6a83ae-fcb1-4b75-9ffa-d52898167d66,cd1de71e-98d1-3cba-8fd2-2b20331ff03f,ff06bdf1-8e9c-3c61-8b34-8bfa8712bed7,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,General examination of patient (procedure),NaN,129.16,...,New Bedford,31.0,Shirley182 Wunsch504,PCP37542,S DARTMOUTH,4003.96,Gabriel934 Tejeda887,M,GENERAL PRACTICE,NO_INSURANCE
32626,48378ab5-e662-44c6-b199-bee936e4894a,2004-06-28T23:35:03Z,2004-06-28T23:50:03Z,475b2a8b-19a0-451a-b965-9c6bebf16434,12c9daf5-a29c-36c9-ac55-28972463e566,aa89beb2-7bc6-35fa-83f7-4b32039e84eb,7caa7254-5050-3b5e-9eae-bd5ea30e809c,Follow-up encounter,Hyperlipidemia,129.16,...,NaN,NaN,NaN,BETH ISRAEL DEACONESS HOSPITAL-MILTON INC,MILTON,253308.58,Sanford861 Gottlieb798,M,GENERAL PRACTICE,Medicare
39168,8df0e941-66e8-4561-b0d4-9d6b8d80b6d1,2011-04-01T12:13:45Z,2011-04-01T12:28:45Z,9384b3da-e034-46fe-bcca-dc19216662c2,24cb4eab-6166-3530-bddc-a5a8a14a4fc1,7bd4e666-a82d-3ad1-bc7c-b49eb726577b,4d71f845-a6a9-3c39-b242-14d25ef86a8d,Encounter for symptom,Viral sinusitis (disorder),129.16,...,Bellingham,17.0,Wes853 Kessler503,MILFORD REGIONAL MEDICAL CENTER,MILFORD,336022.50,Lonna614 Dietrich576,F,GENERAL PRACTICE,Aetna


### Lets clean up the Encounter Table a little bit

In [26]:
#LIST THE COLUMNS IN ENCOUNTER
encounter.columns

Index(['Id', 'START', 'STOP', 'PATIENT_ID', 'ORG_ID', 'PROVIDER_ID',
       'PAYER_ID', 'DESCRIPTION', 'REASON', 'BASE_ENCOUNTER_COST',
       'PAYER_COVERAGE', 'ENCOUNTERCLASS', 'BIRTHDATE', 'CITY', 'AGE',
       'PATIENT_NAME', 'ORG_NAME', 'ORG_CITY', 'REVENUE', 'PROVIDER_NAME',
       'GENDER', 'SPECIALITY', 'PAYER_NAME'],
      dtype='object')

In [27]:
# only select a few columns in it



encounter = encounter[["Id","START","STOP","PATIENT_NAME","AGE","CITY",
                     "DESCRIPTION","REASON","ENCOUNTERCLASS",
                     "BASE_ENCOUNTER_COST","PAYER_COVERAGE",
                     "ORG_NAME","ORG_CITY","REVENUE",
                     "PROVIDER_NAME","SPECIALITY","PAYER_NAME"]]

encounter.sample(3)

,Id,START,STOP,PATIENT_NAME,AGE,CITY,DESCRIPTION,REASON,ENCOUNTERCLASS,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ORG_NAME,ORG_CITY,REVENUE,PROVIDER_NAME,SPECIALITY,PAYER_NAME
42469,b80fdb41-9cf4-4287-951a-200c123d79da,2012-07-03T00:54:28Z,2012-07-03T01:24:28Z,Richard937 Rice937,76.0,Greenfield,Domiciliary or rest home patient evaluation an...,NaN,ambulatory,129.16,29.16,BAYSTATE FRANKLIN MEDICAL CENTER,GREENFIELD,137761.90,Von197 Mraz590,GENERAL PRACTICE,Dual Eligible
37997,101b1534-a86c-4c16-82b9-5c6c3f29bd8a,2018-05-14T19:46:25Z,2018-05-14T20:01:25Z,Stanford577 Stanton715,27.0,Holyoke,Encounter for check up (procedure),NaN,outpatient,129.16,0.00,HOLYOKE MEDICAL CENTER,HOLYOKE,327601.37,Garth972 Wyman904,GENERAL PRACTICE,NO_INSURANCE
17197,6225b710-4ed6-479d-94b5-c709c1243d1a,2010-10-13T00:59:10Z,2010-10-13T01:14:10Z,Gale827 Reynolds644,80.0,Newburyport,General examination of patient (procedure),NaN,wellness,129.16,129.16,PCP661,NEWBURY,14078.44,Phil587 Grimes165,GENERAL PRACTICE,Medicare


In [ ]:
# LETS CREATE A COLUMN YEAR THAT CALCULATES THE YEAR FROM 'START'

In [28]:
encounter["START"]=pd.to_datetime(encounter["START"],format="%Y-%m-%d")

encounter["YEAR"] = encounter["START"].apply(lambda x:x.year)

encounter.sample(3)

/home/riz-ryu/miniconda3/envs/stm/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/riz-ryu/miniconda3/envs/stm/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Id,START,STOP,PATIENT_NAME,AGE,CITY,DESCRIPTION,REASON,ENCOUNTERCLASS,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ORG_NAME,ORG_CITY,REVENUE,PROVIDER_NAME,SPECIALITY,PAYER_NAME,YEAR
28656,7036679e-354d-4315-b982-3f8f1afd35fe,2011-10-24 17:42:20+00:00,2011-10-24T17:57:20Z,Wilber603 Schinner682,70.0,Lynn,Telemedicine consultation with patient,Chronic congestive heart failure (disorder),ambulatory,129.16,54.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,Blue Cross Blue Shield,2011
14034,91d4618e-69e8-45a9-8a9f-fe9f219548f6,1993-04-20 09:14:06+00:00,1993-04-20T09:29:06Z,Ellamae709 Bins636,58.0,Ware,General examination of patient (procedure),NaN,wellness,129.16,69.16,RADIOLOGY AND IMAGING INC.,WARE,8395.40,Lilla884 Marvin195,GENERAL PRACTICE,Cigna Health,1993
42944,5084589f-fdfa-4e23-9b99-3cde72586195,1994-02-17 07:21:57+00:00,1994-02-17T07:36:57Z,Sam879 Corkery305,66.0,Swansea,Encounter for check up (procedure),NaN,outpatient,129.16,0.00,SOUTHCOAST HOSPITAL GROUP INC,FALL RIVER,222412.59,Myong12 Heidenreich818,GENERAL PRACTICE,NO_INSURANCE,1994


In [29]:
encounter["YEAR"].value_counts()

2014    3178
2017    3153
2016    3148
2018    3090
2015    3063
        ... 
1929       2
1918       2
1919       2
1922       2
1913       1
Name: YEAR, Length: 108, dtype: int64

In [30]:
# SELECT ROWS WHERE YEAR >= 2010
encounter = encounter[encounter["YEAR"]>=2010]

In [31]:
encounter.to_csv("consolidated.csv")

In [62]:
encounter=pd.read_csv("https://raw.githubusercontent.com/vedashree29296/healthcare-eda/master/consolidated.csv",index_col=0)

In [63]:
encounter.sample(3)

,Id,START,STOP,PATIENT_NAME,AGE,CITY,DESCRIPTION,REASON,ENCOUNTERCLASS,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ORG_NAME,ORG_CITY,REVENUE,PROVIDER_NAME,SPECIALITY,PAYER_NAME,YEAR
49941,01316031-e42b-40f2-ae10-f36b547d47b6,2016-08-24 13:40:53+00:00,2016-08-24T13:55:53Z,Ali918 Maggio310,43.0,Boston,Encounter for problem (procedure),NaN,outpatient,129.16,69.16,CAMBRIDGE HEALTH ALLIANCE,CAMBRIDGE,406053.19,Jeanmarie510 Beatty507,GENERAL PRACTICE,Medicaid,2016
43374,8a96f27d-ddcb-4a0e-969a-1f1fab7a1e85,2014-10-12 13:46:06+00:00,2014-10-12T14:01:06Z,Virginia437 Calvillo618,29.0,Boston,Encounter for problem (procedure),NaN,outpatient,129.16,69.16,BOSTON CHILDREN'S HOSPITAL,BOSTON,51793.16,Tisha655 Dickens475,GENERAL PRACTICE,Medicaid,2014
44156,f4d493e8-2a81-4ce1-a1b7-3728721e639f,2010-11-17 12:00:58+00:00,2010-11-17T12:30:58Z,Marica887 Casper496,54.0,Lynn,General examination of patient (procedure),NaN,wellness,129.16,129.16,PCP131358,SAUGUS,5553.88,Stephenie799 Vandervort697,GENERAL PRACTICE,Medicaid,2010


#### Lets answer certain general questions for now

### I - How has the year wise trend been with the Insurance companies ? 

We're going to use a simple animation using plotly

In [66]:
# GROUP BY PAYER NAME AND YEAR AND COUNT
counts = ____________________________________

# NOW X-AXIS = insurance companies , Y=AXIS = counts , FRAME BASED ON = year?

px.bar(counts,x=_______________,y=_______________,animation_frame=____________)

In [70]:
# GROUP BY YEAR AND PAYER_NAME AND COUNT
counts = _____________________________

# NOW X-AXIS = insurance companies , Y=AXIS = counts , FRAME BASED ON = year?

px.bar(counts,x=_____________,y=_____________,color=__________________,barmode=___________)

NameError: name '_____________________________' is not defined

In [ ]:
# Simple subplot of this
# payers: 
rows=
cols=
specs=

fig=make_subplots()

In [ ]:
#grouping

### II - How has the year wise trend been in the number of patient visits ?

This is bit more complex animation with a play button

In [38]:
# GROUP BY YEAR AND COUNT (ALSO RESET THE INDEX)
no_cases=encounter.groupby("YEAR").count().reset_index()
no_cases

,YEAR,Id,START,STOP,PATIENT_NAME,AGE,CITY,DESCRIPTION,REASON,ENCOUNTERCLASS,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ORG_NAME,ORG_CITY,REVENUE,PROVIDER_NAME,SPECIALITY,PAYER_NAME
0,2010,2083,2083,2083,1999,1999,1999,2083,609,2083,2083,2083,2083,2083,2083,2083,2083,2083
1,2011,2979,2979,2979,2885,2885,2885,2979,1012,2979,2979,2979,2979,2979,2979,2979,2979,2979
2,2012,2890,2890,2890,2806,2806,2806,2890,947,2890,2890,2890,2890,2890,2890,2890,2890,2890
3,2013,3058,3058,3058,2966,2966,2966,3058,950,3058,3058,3058,3058,3058,3058,3058,3058,3058
4,2014,3178,3178,3178,2948,2948,2948,3178,1059,3178,3178,3178,3178,3178,3178,3178,3178,3178
5,2015,3063,3063,3063,2964,2964,2964,3063,943,3063,3063,3063,3063,3063,3063,3063,3063,3063
6,2016,3148,3148,3148,3064,3064,3064,3148,967,3148,3148,3148,3148,3148,3148,3148,3148,3148
7,2017,3153,3153,3153,3064,3064,3064,3153,991,3153,3153,3153,3153,3153,3153,3153,3153,3153
8,2018,3090,3090,3090,3040,3040,3040,3090,910,3090,3090,3090,3090,3090,3090,3090,3090,3090
9,2019,2919,2919,2919,2867,2867,2867,2919,774,2919,2919,2919,2919,2919,2919,2919,2919,2919


#### We need to create a year wise trend line animation

- x- axis: ?

- y-axis : ?

For this we need : 
- range of values on the x axis and y axis-/
- starting values for x and y
- x and y values for each year

And for the plotly chart we need to define:
- Layout (Buttons)
- Frames (x and y value of each year) 
- data: starting and y values = /

In [42]:
# find the range of x values and y values

x_range = [no_cases["YEAR"].min(),no_cases["YEAR"].max()]

y_range = [no_cases["Id"].min(),no_cases["Id"].max()]

In [43]:
# Starting x values and start y values
start_x = no_cases["YEAR"].min()

start_y = no_cases["Id"].min()

data = go.Scatter(x=[start_x,start_x],y=[start_y,start_y])


In [55]:
# individual x and y values

frames = []

for i in range(1,len(no_cases)+1):
    #i = 2013
    # : i upto 2013 (2010,2011,2012)
    frame = go.Scatter(x=no_cases["YEAR"].iloc[:i],y=no_cases["Id"].iloc[:i])
    frame = go.Frame(data=[frame])
    frames.append(frame)
    


In [56]:
button = {"label":"Play",
          "method":"animate",
         "args":[None]
         }

update_menus = {"type":"buttons",
         "buttons":[button],
          }

layout=go.Layout(xaxis=dict(range=x_range),yaxis=dict(range=y_range),updatemenus=[update_menus])


In [57]:
fig=go.Figure(data=[data],layout=layout,frames=frames)
fig.show()

## Now consider the stakeholders involved in this scenario, and try to determine what analysis of data  will be useful for each of them.

#### Lets take the example of the hospitals here. What all data can be useful for one hospital ?
The hospital has data records for the past 10 years. Thequestions asked can include
- How many patients came in each year ? 
- What is the most popular reason for a patient to visit ?
- Which insurance company is the most popular ?
- What are the common medications/tests prescribed ? 
- What doctors/ speciality is most in demand ?
- How many emergency cases are coming in ?
- Whats the age group of the patients ?

Depending on the answers to these questions 
the hospital can take decisions like:
- What speciality of doctors need to be hired ? 
- What medicines/ medical equipment should be ordered ? 
- Which Insurance company can be a potential partner ? 
- How stringent should the emergency preparedness be ?

### Lets look at how many hospitals do we have

In [104]:
encounter["ORG_NAME"].value_counts()

HALLMARK HEALTH SYSTEM                               1324
VA Boston Healthcare System  Jamaica Plain Campus    1196
METROWEST MEDICAL CENTER                              712
NEWTON-WELLESLEY HOSPITAL                             691
CAMBRIDGE HEALTH ALLIANCE                             689
                                                     ... 
HAWTHORN MEDICAL URGENT CARE CENTER                     2
HAMPDEN COUNTY URGENT MEDICAL CARE CENTER               2
MASHPEE FAMILY MEDICINE                                 1
TRU MEDICAL WALK IN CENTER                              1
FALL RIVER MEDICAL EMERGENCY WALK IN CLINIC             1
Name: ORG_NAME, Length: 883, dtype: int64

### Lets choose a random hospital. We're going with HALLMARK HEALTHCARE

In [58]:
df=encounter[encounter["ORG_NAME"]=="HALLMARK HEALTH SYSTEM"]
df

,Id,START,STOP,PATIENT_NAME,AGE,CITY,DESCRIPTION,REASON,ENCOUNTERCLASS,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ORG_NAME,ORG_CITY,REVENUE,PROVIDER_NAME,SPECIALITY,PAYER_NAME,YEAR
632,0f2eccff-7741-4ff2-a13a-656cbd78394a,2010-12-17 14:19:13+00:00,2010-12-17T19:34:13Z,Lisbeth69 Hand679,37.0,Malden,Prenatal initial visit,Normal pregnancy,ambulatory,129.16,64.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,Aetna,2010
633,a8f5aec6-5b20-4c6e-b79b-4adeac18f978,2011-01-14 14:19:13+00:00,2011-01-14T15:19:13Z,Lisbeth69 Hand679,37.0,Malden,Prenatal visit,Normal pregnancy,ambulatory,129.16,64.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,Aetna,2011
634,1bd80074-32d8-454f-916a-4aff32595a81,2011-02-11 14:19:13+00:00,2011-02-11T15:34:13Z,Lisbeth69 Hand679,37.0,Malden,Prenatal visit,Normal pregnancy,ambulatory,129.16,64.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,Aetna,2011
635,2e9c5abc-f9d1-45fa-9a98-69f99fb245d9,2011-03-11 14:19:13+00:00,2011-03-11T15:04:13Z,Lisbeth69 Hand679,37.0,Malden,Prenatal visit,Normal pregnancy,ambulatory,129.16,64.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,Aetna,2011
636,61b102d3-fa48-49a8-b6bc-9214a052b6df,2011-04-08 14:19:13+00:00,2011-04-08T15:04:13Z,Lisbeth69 Hand679,37.0,Malden,Prenatal visit,Normal pregnancy,ambulatory,129.16,64.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,Aetna,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52475,eb1b8c76-56f4-46ea-9e5f-5f0363db9858,2013-01-07 12:14:09+00:00,2013-01-07T12:29:09Z,Hubert238 Kihn564,12.0,Boston,Encounter for check up (procedure),NaN,outpatient,129.16,0.00,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,UnitedHealthcare,2013
52476,fbb8bdc8-f506-4c1f-b899-0da2b4859a1a,2013-01-06 12:14:09+00:00,2013-01-06T12:29:09Z,Hubert238 Kihn564,12.0,Boston,Encounter for symptom,Acute bacterial sinusitis (disorder),ambulatory,129.16,59.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,UnitedHealthcare,2013
52478,b35110b5-20b7-4ebf-84c5-62bf97ed94a3,2013-08-14 12:14:09+00:00,2013-08-14T12:29:09Z,Hubert238 Kihn564,12.0,Boston,Encounter for symptom,Viral sinusitis (disorder),ambulatory,129.16,59.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,UnitedHealthcare,2013
52479,9a384fbd-3e24-41e6-b338-330a6efdddeb,2013-12-27 12:14:09+00:00,2013-12-27T12:29:09Z,Hubert238 Kihn564,12.0,Boston,Encounter for symptom,Viral sinusitis (disorder),ambulatory,129.16,59.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,UnitedHealthcare,2013


#### Lets do a simple visualisation. 
We'd like to know what was the distribution of the number of cases covered by each insurance agency
Lets create a stacked chart

In [136]:
#group by year and payer name and take a count.
payer_distribution=__________________________________

# x-axis = ? y-axis=? data to be seen = ?

px.bar(payer_distribution,x=___________,y=___________,hover_data=[__________],labels={"Id":"num_visits"})

#### Now let's try to find the popular reasons for which patients visited the hospital
We're going to use subplots for this
We need: 
- number of rows 
- number of columns
- and what each figure is going be (specs)

In [151]:
cols= 1
rows = 11

specs = [[{"type":"pie"}] for i in range(rows)]

column_widths=[0.9]

In [152]:
fig = make_subplots(rows=rows,cols=cols,specs=specs,column_widths=column_widths)

In [153]:
row=1
# first group by year and then for each group we're going to take the value counts for the reason
for year,group in ______________________:
    counts=______________________
    
    fig.append_trace(go.Pie(title=,labels=counts._________,values=counts._________,textinfo="none"),row=____,col=1)
    row = _____________
    
fig.update_layout(height=4000, width=1000)
fig.show()

#### Now can we apply the same to find out the age distribution of the patients in every year?

In [154]:
row=1
fig=make_subplots(___________________________)

# group by year and iterate over it
# how about using an apply function here ?

for year,group in _____________________:
    
    fig.append_trace(go.Histogram(x=__________,name=___________),row=row,col=1)
    row+=1
fig.update_layout(height=2000,width=1000)
fig.show()

#### Practise:  Now try finding the number of visits from each city in each year (Lets just do it for 5 cities)

### Takeaways and assignments:
Use the same logic to analyse and create visualisations from the point of view of some other stakeholder. Think about other possibilities. Get creat